In [1]:
# Copyright 2019-2022 ETH Zurich and the DaCe authors. All rights reserved.
"""
Sample showcasing transformations applied on a naive matrix multiplication program, yielding performance that competes
with Intel MKL and NVIDIA CUBLAS.
"""
import click
import dace
import numpy as np
from typing import List, Tuple

# For optimizations
from dace.transformation.dataflow import (DoubleBuffering, MapCollapse, MapExpansion, MapReduceFusion, StripMining,
                                          InLocalStorage, AccumulateTransient, Vectorization)
from dace.transformation.interstate import FPGATransformSDFG
from dace.transformation import helpers as xfutil

# For library node implementations
import dace.libraries.blas

# Define symbolic sizes for arbitrary inputs
M = dace.symbol('M')
K = dace.symbol('K')
N = dace.symbol('N')

# Define data type to use
dtype = dace.float64
np_dtype = np.float64

#####################################################################
# Data-centric functions


# Map-Reduce version of matrix multiplication
@dace.program
def matmul(A: dtype[M, K], B: dtype[K, N], C: dtype[M, N]):
    tmp = np.ndarray([M, N, K], dtype=A.dtype)

    # Multiply every pair of values to a large 3D temporary array
    for i, j, k in dace.map[0:M, 0:N, 0:K]:
        with dace.tasklet:
            in_A << A[i, k]
            in_B << B[k, j]
            out >> tmp[i, j, k]

            out = in_A * in_B

    # Sum last dimension of temporary array to obtain resulting matrix
    dace.reduce(lambda a, b: a + b, tmp, C, axis=2, identity=0)


# Library node version of matrix multiplication, using the numpy interface
@dace.program
def matmul_lib(A: dtype[M, K], B: dtype[K, N]):
    return A @ B


#####################################################################
# Data-centric optimization helpers


def find_map_by_param(sdfg: dace.SDFG, pname: str) -> dace.nodes.MapEntry:
    """ Finds the first map entry node by the given parameter name. """
    return next(n for n, _ in sdfg.all_nodes_recursive() if isinstance(n, dace.nodes.MapEntry) and pname in n.params)


def find_map_and_state_by_param(sdfg: dace.SDFG, pname: str) -> Tuple[dace.nodes.MapEntry, dace.SDFGState]:
    """ Finds the first map entry node by the given parameter name. """
    return next(
        (n, p) for n, p in sdfg.all_nodes_recursive() if isinstance(n, dace.nodes.MapEntry) and pname in n.params)


def find_mapexit_by_param(sdfg: dace.SDFG, pname: str) -> dace.nodes.MapExit:
    """ Finds the first map exit node by the given parameter name. """
    entry, state = find_map_and_state_by_param(sdfg, pname)
    return state.exit_node(entry)


#####################################################################
# Matrix multiplication data-centric optimization schemes


def optimize_for_cpu(sdfg: dace.SDFG, m: int, n: int, k: int):
    """ Optimize the matrix multiplication example for multi-core CPUs. """
    # Ensure integers are 32-bit by default
    dace.Config.set('compiler', 'default_data_types', value='C')

    # Fuse the map and reduce nodes
    sdfg.apply_transformations(MapReduceFusion)

    # Find multiplication map
    entry = find_map_by_param(sdfg, 'k')

    # Create a tiling strategy
    divides_evenly = (m % 32 == 0) and (n % 32 == 0) and (k % 256 == 0)
    xfutil.tile(sdfg, entry, divides_evenly, False, k=256, i=32, j=32)
    xfutil.tile(sdfg, entry, divides_evenly, divides_evenly, j=16, i=4)

    # Reorder internal map to "k,i,j"
    xfutil.permute_map(entry, [2, 0, 1])

    # Add local storage for B in j tile: we apply InLocalStorage with a
    # parameter "array" named B, between the two maps of j and i
    regtile_j = find_map_by_param(sdfg, 'tile1_j')
    regtile_i = find_map_by_param(sdfg, 'tile1_i')
    InLocalStorage.apply_to(sdfg, dict(array='B'), node_a=regtile_j, node_b=regtile_i)

    if divides_evenly:
        # Add local storage for C
        exit_inner = find_mapexit_by_param(sdfg, 'k')
        exit_rti = find_mapexit_by_param(sdfg, 'tile1_i')
        AccumulateTransient.apply_to(sdfg, dict(array='C', identity=0), map_exit=exit_inner, outer_map_exit=exit_rti)

        # Vectorize microkernel map
        postamble = n % 4 != 0
        entry_inner, inner_state = find_map_and_state_by_param(sdfg, 'k')
        Vectorization.apply_to(inner_state.parent,
                               dict(vector_len=4, preamble=False, postamble=postamble),
                               map_entry=entry_inner)

    # Mark outer tile map as sequential to remove atomics
    find_map_by_param(sdfg, 'tile_k').map.schedule = dace.ScheduleType.Sequential

    # Collapse maps for more parallelism
    find_map_by_param(sdfg, 'o0').map.collapse = 2
    tile_i = find_map_by_param(sdfg, 'tile_i')
    tile_j = find_map_by_param(sdfg, 'tile_j')
    MapCollapse.apply_to(sdfg, outer_map_entry=tile_i, inner_map_entry=tile_j)
    tile_ij = find_map_by_param(sdfg, 'tile_i')  # Find newly created map
    tile_ij.map.schedule = dace.ScheduleType.CPU_Multicore
    tile_ij.map.collapse = 2


def optimize_for_gpu_double_buffer(sdfg: dace.SDFG, m: int, n: int, k: int):
    """ Optimize the matrix multiplication example for GPUs. """
    # Ensure integers are 32-bit by default
    dace.Config.set('compiler', 'default_data_types', value='C')

    # Fuse the map and reduce nodes
    sdfg.apply_transformations(MapReduceFusion)

    # Apply GPU transformation
    sdfg.apply_gpu_transformations()

    # Find multiplication map
    entry = find_map_by_param(sdfg, 'k')

    # Create a tiling strategy
    divides_evenly = (m % 64 == 0) and (n % 64 == 0) and (k % 8 == 0)
    xfutil.tile(sdfg, entry, divides_evenly, True, i=64, j=64, k=8)
    xfutil.tile(sdfg, entry, divides_evenly, True, i=8, j=4)

    # Create kernel schedule by collapsing and reordering maps
    gtile_i = find_map_by_param(sdfg, 'tile_i')
    gtile_j = find_map_by_param(sdfg, 'tile_j')
    btile_i = find_map_by_param(sdfg, 'tile1_i')
    btile_j = find_map_by_param(sdfg, 'tile1_j')
    MapCollapse.apply_to(sdfg, outer_map_entry=gtile_i, inner_map_entry=gtile_j, permissive=True)
    MapCollapse.apply_to(sdfg, outer_map_entry=btile_i, inner_map_entry=btile_j, permissive=True)
    btile = find_map_by_param(sdfg, 'tile1_i')
    btile.map.schedule = dace.ScheduleType.GPU_ThreadBlock

    # Add local storage (shared memory) for A and B on GPU
    ktile = find_map_by_param(sdfg, 'tile_k')
    smem_a = InLocalStorage.apply_to(sdfg, dict(array='A'), node_a=ktile, node_b=btile)
    smem_b = InLocalStorage.apply_to(sdfg, dict(array='B'), node_a=ktile, node_b=btile)
    sdfg.arrays[smem_a.data].storage = dace.StorageType.GPU_Shared
    sdfg.arrays[smem_b.data].storage = dace.StorageType.GPU_Shared

    # Add local storage (registers) for A and B
    ttile = find_map_by_param(sdfg, 'k')
    warptile, ttile = xfutil.extract_map_dims(sdfg, ttile, [2])
    InLocalStorage.apply_to(sdfg, dict(array='trans_gpu_A'), node_a=warptile, node_b=ttile)
    InLocalStorage.apply_to(sdfg, dict(array='trans_gpu_B'), node_a=warptile, node_b=ttile)

    # Add local storage (registers) for C
    state = next(s for s in sdfg.nodes() if warptile in s.nodes())
    warptile_exit = state.exit_node(warptile)
    btile_exit = state.exit_node(btile)
    AccumulateTransient.apply_to(sdfg, map_exit=warptile_exit, outer_map_exit=btile_exit)
    # Set C tile to zero on allocation
    c_access = next(n for n in state.data_nodes() if n.data == 'trans_gpu_C')
    c_access.setzero = True

    # Unroll microkernel maps
    ttile.map.unroll = True

    # Apply double-buffering on shared memory
    DoubleBuffering.apply_to(sdfg, map_entry=ktile, transient=smem_a)

def optimize_for_gpu(sdfg: dace.SDFG, m: int, n: int, k: int):
    """ Optimize the matrix multiplication example for GPUs. """
    # Ensure integers are 32-bit by default
    dace.Config.set('compiler', 'default_data_types', value='C')

    # Fuse the map and reduce nodes
    sdfg.apply_transformations(MapReduceFusion)

    # Apply GPU transformation
    sdfg.apply_gpu_transformations()

    # Find multiplication map
    entry = find_map_by_param(sdfg, 'k')

    # Create a tiling strategy
    divides_evenly = (m % 64 == 0) and (n % 64 == 0) and (k % 8 == 0)
    xfutil.tile(sdfg, entry, divides_evenly, True, i=64, j=64, k=8)
    xfutil.tile(sdfg, entry, divides_evenly, True, i=8, j=4)

    # Create kernel schedule by collapsing and reordering maps
    gtile_i = find_map_by_param(sdfg, 'tile_i')
    gtile_j = find_map_by_param(sdfg, 'tile_j')
    btile_i = find_map_by_param(sdfg, 'tile1_i')
    btile_j = find_map_by_param(sdfg, 'tile1_j')
    MapCollapse.apply_to(sdfg, outer_map_entry=gtile_i, inner_map_entry=gtile_j, permissive=True)
    MapCollapse.apply_to(sdfg, outer_map_entry=btile_i, inner_map_entry=btile_j, permissive=True)
    btile = find_map_by_param(sdfg, 'tile1_i')
    btile.map.schedule = dace.ScheduleType.GPU_ThreadBlock

    # Add local storage (shared memory) for A and B on GPU
    ktile = find_map_by_param(sdfg, 'tile_k')
    smem_a = InLocalStorage.apply_to(sdfg, dict(array='A'), node_a=ktile, node_b=btile)
    smem_b = InLocalStorage.apply_to(sdfg, dict(array='B'), node_a=ktile, node_b=btile)
    sdfg.arrays[smem_a.data].storage = dace.StorageType.GPU_Shared
    sdfg.arrays[smem_b.data].storage = dace.StorageType.GPU_Shared

    # Add local storage (registers) for A and B
    ttile = find_map_by_param(sdfg, 'k')
    warptile, ttile = xfutil.extract_map_dims(sdfg, ttile, [2])
    InLocalStorage.apply_to(sdfg, dict(array='trans_gpu_A'), node_a=warptile, node_b=ttile)
    InLocalStorage.apply_to(sdfg, dict(array='trans_gpu_B'), node_a=warptile, node_b=ttile)

    # Add local storage (registers) for C
    state = next(s for s in sdfg.nodes() if warptile in s.nodes())
    warptile_exit = state.exit_node(warptile)
    btile_exit = state.exit_node(btile)
    AccumulateTransient.apply_to(sdfg, map_exit=warptile_exit, outer_map_exit=btile_exit)
    # Set C tile to zero on allocation
    c_access = next(n for n in state.data_nodes() if n.data == 'trans_gpu_C')
    c_access.setzero = True

    # Unroll microkernel maps
    ttile.map.unroll = True
#####################################################################
# Main function


# @click.command()
# @click.option('-M', type=int, default=64)
# @click.option('-K', type=int, default=64)
# @click.option('-N', type=int, default=64)
# @click.option('--version',
#               type=click.Choice(('unoptimized', 'optimize_cpu', 'optimize_gpu', 'mkl', 'cublas', 'rocblas',
#                                  'fpga_naive', 'fpga_library')),
#               default='unoptimized')
@click.option('--verify/--no-verify', default=True)
def cli(m, k, n, version, verify):
    """
    Different available versions:
    unoptimized: Run `matmul` without optimizations;
    optimize_cpu: Transform `matmul` to a reasonably-optimized version for
                    multicore CPU;
    optimize_gpu: Transform `matmul` to a reasonably-optimized version for GPU;
    mkl: Use `matmul_lib` with the MKL library node implementation;
    cublas: Use `matmul_lib` with the CUBLAS library node implementation.
    """

    # Prepare data with numpy
    A = np.random.rand(m, k).astype(np_dtype)
    B = np.random.rand(k, n).astype(np_dtype)
    C = np.zeros((m, n), dtype=np_dtype)

    print(f'Matrix multiplication {m}x{k}x{n} (version: {version})')

    if version == 'unoptimized':
        # Simply call the program to run it
        matmul(A, B, C)
    elif version.startswith('optimize_'):
        # Get the SDFG from the program
        sdfg: dace.SDFG = matmul.to_sdfg()
        # Call transformations to optimize
        if version == 'optimize_cpu':
            optimize_for_cpu(sdfg, m, n, k)
        elif version == 'optimize_gpu':
            optimize_for_gpu(sdfg, m, n, k)
        # Invoke the SDFG to run the optimized program (notice that now we must
        # also directly feed in the symbols)
        sdfg(A=A, B=B, C=C, M=m, N=n, K=k)
    elif version == 'mkl':
        # Set default implementation to MKL
        dace.libraries.blas.default_implementation = 'MKL'
        # Call program
        C = matmul_lib(A, B)
    elif version == 'cublas':
        # Set default implementation to CUBLAS
        dace.libraries.blas.default_implementation = 'cuBLAS'
        # Call program
        C = matmul_lib(A, B)
    elif version == 'rocblas':
        # Set default implementation to rocBLAS
        dace.libraries.blas.default_implementation = 'rocBLAS'
        # Call program
        C = matmul_lib(A, B)
    elif version == 'fpga_naive':
        matmul_sdfg = matmul.to_sdfg()
        matmul_sdfg.apply_transformations(FPGATransformSDFG)
        matmul_sdfg(A=A, B=B, C=C, N=n, K=k, M=m)
    elif version == 'fpga_systolic':
        dace.libraries.blas.default_implementation = 'FPGA1DSystolic'
        C = matmul_lib(A, B)
    else:
        raise ValueError('Invalid version %s' % version)

    if verify:
        expected = A @ B
        diff = np.linalg.norm(C - expected) / (m * n)
        print('Difference:', diff)
        return 0 if diff <= 1e-6 else 1




In [ ]:
m, n, k = 64, 64, 64
A = np.random.rand(m, k).astype(np_dtype)
B = np.random.rand(k, n).astype(np_dtype)
C = np.zeros((m, n), dtype=np_dtype)
sdfg: dace.SDFG = matmul.to_sdfg()
optimize_for_gpu(sdfg, m, n, k)
sdfg_double_buffer: dace.SDFG = matmul.to_sdfg()
optimize_for_gpu_double_buffer(sdfg_double_buffer, m, n, k)
sdfg_double_buffer.save('matmul_double_buffer.sdfgz')

/scratch/dace4softhier/dace/dace/transformation/dataflow/local_storage.py:80: UserWarning: Array A not found! Using array gpu_A instead.
  warnings.warn('Array %s not found! Using array %s instead.' % (array, invariant_memlet.data))
/scratch/dace4softhier/dace/dace/transformation/dataflow/local_storage.py:80: UserWarning: Array B not found! Using array gpu_B instead.
  warnings.warn('Array %s not found! Using array %s instead.' % (array, invariant_memlet.data))
/scratch/dace4softhier/dace/dace/transformation/dataflow/stream_transient.py:188: UserWarning: AccumulateTransient did not properly initialize newly-created transient!
  warnings.warn('AccumulateTransient did not properly initialize ' 'newly-created transient!')


SDFG (matmul)

In [ ]:
from IPython.display import Code
Code(sdfg.generate_code()[1].clean_code, language='cpp')
sdfg.compile()

~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047c1a10> in state 2
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047c1a10> in state 2
~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047931d0> in state 2
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047931d0> in state 2
~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047c1690> in state 2
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047c1690> in state 2
~~~~~~~~~~~~~~~Dispatching subgraph block in state 1
~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047796d0> in state 1
Generating Tasklet Using CPU Codegen
Defining Out Memlet
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f90047796d0> i

/scratch/dace4softhier/dace/dace/codegen/targets/cuda.py:1870: UserWarning: No `gpu_block_size` property specified on map "freduce_init_map". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


#include <cuda_runtime.h>
#include <dace/dace.h>


struct matmul_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_cuda(matmul_state_t *__state, int K, int M, int N);
DACE_EXPORTED int __dace_exit_cuda(matmul_state_t *__state);



int __dace_init_cuda(matmul_state_t *__state, int K, int M, int N) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(3, 2);

    // Create cuda streams and events
    for(int i = 0; i < 3; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 2; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_cuda(matmul_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 3; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 2; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(matmul_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 3)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(matmul_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 3; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void __launch_bounds__(32) freduce_init_map_0_1_0(double * __restrict__ gpu_C, int M, int N) {
    {
        {
            int o1 = (blockIdx.x * 32 + threadIdx.x);
            int o0 = (blockIdx.y * 1 + threadIdx.y);
            if (o1 < N) {
                {
                    {
                        double __out;

                        ///////////////////
                        // Tasklet code (freduce_init)
                        __out = 0;
                        ///////////////////

                        gpu_C[((N * o0) + o1)] = __out;
                    }
                }
            }
        }
    }
}


DACE_EXPORTED void __dace_runkernel_freduce_init_map_0_1_0(matmul_state_t *__state, double * __restrict__ gpu_C, int M, int N);
void __dace_runkernel_freduce_init_map_0_1_0(matmul_state_t *__state, double * __restrict__ gpu_C, int M, int N)
{

    if ((int_ceil(int_ceil(N, 1), 32)) == 0 || (int_ceil(int_ceil(M, 1), 1)) == 0) {

        return;
    }

    void  *freduce_init_map_0_1_0_args[] = { (void *)&gpu_C, (void *)&M, (void *)&N };
    gpuError_t __err = cudaLaunchKernel((void*)freduce_init_map_0_1_0, dim3(int_ceil(int_ceil(N, 1), 32), int_ceil(int_ceil(M, 1), 1), 1), dim3(32, 1, 1), freduce_init_map_0_1_0_args, 0, __state->gpu_context->streams[0]);
    DACE_KERNEL_LAUNCH_CHECK(__err, "freduce_init_map_0_1_0", int_ceil(int_ceil(N, 1), 32), int_ceil(int_ceil(M, 1), 1), 1, 32, 1, 1);
}
__global__ void __launch_bounds__(128) matmul_39_0_0_9(const double * __restrict__ gpu_A, const double * __restrict__ gpu_B, double * __restrict

In [ ]:
Code(sdfg_double_buffer.generate_code()[1].clean_code, language='cpp') 
sdfg_double_buffer.compile()

~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9002d062d0> in state 2
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9002d062d0> in state 2
~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9002d07810> in state 2
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9002d07810> in state 2
~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9002d077d0> in state 2
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9002d077d0> in state 2
~~~~~~~~~~~~~~~Dispatching subgraph block in state 1
~~~~~~~~~~~~~~~Dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9004562fd0> in state 1
Generating Tasklet Using CPU Codegen
Defining Out Memlet
~~~~~~~~~~~~~~~Finished dispatching subgraph <dace.sdfg.scope.ScopeSubgraphView object at 0x7f9004562fd0> i

#include <cuda_runtime.h>
#include <dace/dace.h>


struct matmul_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_cuda(matmul_state_t *__state, int K, int M, int N);
DACE_EXPORTED int __dace_exit_cuda(matmul_state_t *__state);

DACE_DFI void nested_MapState_0_0_6(const double * __restrict__ gpu_A, const double * __restrict__ gpu_B, double * __restrict__ gpu_C, int K, int N) {
    double trans_trans_gpu_B[4]  DACE_ALIGN(64);
    double trans_trans_gpu_A[8]  DACE_ALIGN(64);
    __shared__ double trans_gpu_B[1024];
    __shared__ double trans_gpu_A[1024];
    double trans_gpu_C[32]  DACE_ALIGN(64) = {0};
    int tile_k;

    {
        // Start allocate arrays in scope
        // Finish allocate arrays in scope
        dace::GlobalToShared2D<double, 16, 8, 1, 64, 8, 8, 1, true>(gpu_A, K, 1, trans_gpu_A);
        dace::GlobalToShared2D<double, 16, 8, 1, 8, 64, 64, 1, true>(gpu_B, N, 1, trans_gpu_B);

    }
    // Start allocate arrays in scope
    // Finish allocate arrays in scope

    for (tile_k = 0; (tile_k < (K - 8)); tile_k = (tile_k + 8)) {
        {
            // Start allocate arrays in scope
            // Finish allocate arrays in scope
            {
                {
                    {
                        __syncthreads();
                        int tile1_j = (4 * threadIdx.x);
                        int tile1_i = (8 * threadIdx.y);
                        {
                            {
                                {
                                    for (auto k = 0; k < 8; k += 1) {
                                        // CPU: Copy from matmul_39[k=0:8] to trans_trans_gpu_A

                                        dace::CopyND<double, 1, false, 8>::template ConstDst<1>::Copy(
                                        trans_gpu_A + ((k + (8 * tile1_i)) + (512 * ((tile_k / 8) % 2))), trans_trans_gpu_A, 8);
                                        // CPU: Copy from matmul_39[k=0:8] to trans_trans_gpu_B

                                        dace::CopyND<double, 1, false, 4>::template ConstDst<1>::Copy(
                                        trans_gpu_B + (((64 * k) + tile1_j) + (512 * ((tile_k / 8) % 2))), trans_trans_gpu_B, 1);
                                        {
                                            #pragma unroll
                                            for (auto i = 0; i < 8; i += 1) {
                                                #pragma unroll
                                                for (auto j = 0; j < 4; j += 1) {
                                                    {
                                                        // CPU: Copy from matmul_39_i[i=0:8, j=0:4] to matmul_40
                                                        double in_A = trans_trans_gpu_A[i];
                                                        // CPU: Copy from matmul_39_i[i=0:8, j=0:4] to matmul_40
                                                        double in_B = trans_trans_gpu_B[j];
                                                        double out;

                                                        ///////////////////
                                                        // Tasklet code (matmul_40)
                                                        out = (in_A * in_B);
                                                        ///////////////////

                                                        dace::wcr_fixed<dace::ReductionType::Sum, double>::reduce(trans_gpu_C + ((4 * i) + j), out);
                                                    }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
            dace::GlobalToShared2D<double, 16, 8, 1, 64, 8, 8, 1, true>(gpu_A + (